# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [141]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [142]:
# all the cleaning

df = df.drop(columns="customerID")

In [143]:
df.gender = df.gender.replace({"Female": 1, "Male": 0})

In [144]:
df.Partner = df.Partner.replace({"No": 0, "Yes": 1})

In [145]:
df.Dependents = df.Dependents.replace({"No": 0, "Yes": 1})

In [146]:
df.PhoneService = df.PhoneService.replace({"No": 0, "Yes": 1})

In [147]:
df.MultipleLines = df.MultipleLines.replace({"No": 0, "Yes": 1, "No phone service": 0})

In [148]:
df.InternetService = df.InternetService.replace({"No": 0, "DSL": 1, "Fiber optic": 2})

In [149]:
df.OnlineSecurity = df.OnlineSecurity.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [150]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [151]:
df.OnlineBackup = df.OnlineBackup.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [152]:
df.DeviceProtection = df.DeviceProtection.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [153]:
df.TechSupport = df.TechSupport.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [154]:
df.StreamingTV = df.StreamingTV.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [155]:
df.StreamingMovies = df.StreamingMovies.replace({"No": 0, "Yes": 1, "No internet service": 0})

In [156]:
df.Contract = df.Contract.replace({"Month-to-month": 0, "One year": 1, "Two year": 2})

In [157]:
df.PaperlessBilling = df.PaperlessBilling.replace({"No": 0, "Yes": 1})

In [158]:
df.PaymentMethod = df.PaymentMethod.replace({"Electronic check": 3, 
                                             "Mailed check": 2, 
                                             "Bank transfer (automatic)": 1, 
                                             "Credit card (automatic)": 0})

In [159]:
import numpy as np

df.TotalCharges = df.TotalCharges.replace({" ": np.nan})

In [160]:
df.TotalCharges.isnull().sum()

11

In [161]:
df.TotalCharges = df.TotalCharges.dropna()

In [162]:
df.TotalCharges = pd.to_numeric(df.TotalCharges)

In [163]:
df.Churn = df.Churn.replace({"No": 0, "Yes": 1})

In [164]:
df.describe().shape

(8, 20)

In [165]:
# get train test sets

from sklearn.model_selection import train_test_split

X = df.drop(columns="Churn")
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [166]:
# normalize data

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.fit_transform(X_test, y_test)

/Users/ljohnson/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [169]:
# set outputs to categorical

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

In [170]:
y_train[0:10]

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
# get baseline model going

import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(19, input_dim=19, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation="sigmoid"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, validation_split=.2, epochs=10, batch_size=10)

Train on 3774 samples, validate on 944 samples
Epoch 1/10
3774/3774 [==============================] - 3s 771us/step - loss: nan - acc: 0.7130 - val_loss: nan - val_acc: 0.7553
Epoch 2/10
3774/3774 [==============================] - 1s 266us/step - loss: nan - acc: 0.7324 - val_loss: nan - val_acc: 0.7553
Epoch 3/10
3774/3774 [==============================] - 1s 265us/step - loss: nan - acc: 0.7324 - val_loss: nan - val_acc: 0.7553
Epoch 4/10
1680/3774 [============>.................] - ETA: 0s - loss: nan - acc: 0.7196

In [133]:
# trying some tuning of batch size

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=19, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

model = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/20
4718/4718 [==============================] - 1s 119us/step - loss: nan - acc: 0.7149
Epoch 2/20
4718/4718 [==============================] - 0s 19us/step - loss: nan - acc: 0.7370
Epoch 3/20
4718/4718 [==============================] - 0s 18us/step - loss: nan - acc: 0.7370
Epoch 4/20
4718/4718 [==============================] - 0s 20us/step - loss: nan - acc: 0.7370
Epoch 5/20
4718/4718 [==============================] - 0s 18us/step - loss: nan - acc: 0.7370
Epoch 6/20
4718/4718 [==============================] - 0s 20us/step - loss: nan - acc: 0.7370
Epoch 7/20
4718/4718 [==============================] - 0s 18us/step - loss: nan - acc: 0.7370
Epoch 8/20
4718/4718 [==============================] - 0s 21us/step - loss: nan - acc: 0.7370
Epoch 9/20
4718/4718 [==============================] - 0s 24us/step - loss: nan - acc: 0.7370
Epoch 10/20
4718/4718 [==============================] - 0s 28us/step - loss: nan - acc: 0.7370
Epoch 11/20
4718/4718 [=========================

In [140]:
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(64, input_dim=19, activation=activation))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

model = KerasClassifier(build_fn=create_model, batch_size=100, epochs=20, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/20
3145/3145 [==============================] - 1s 184us/step - loss: nan - acc: 0.7345
Epoch 2/20
3145/3145 [==============================] - 0s 20us/step - loss: nan - acc: 0.7345
Epoch 3/20
3145/3145 [==============================] - 0s 25us/step - loss: nan - acc: 0.7345
Epoch 4/20
3145/3145 [==============================] - 0s 22us/step - loss: nan - acc: 0.7345
Epoch 5/20
3145/3145 [==============================] - 0s 23us/step - loss: nan - acc: 0.7345
Epoch 6/20
3145/3145 [==============================] - 0s 26us/step - loss: nan - acc: 0.7345
Epoch 7/20
3145/3145 [==============================] - 0s 22us/step - loss: nan - acc: 0.7345
Epoch 8/20
3145/3145 [==============================] - 0s 22us/step - loss: nan - acc: 0.7345
Epoch 9/20
3145/3145 [==============================] - 0s 24us/step - loss: nan - acc: 0.7345
Epoch 10/20
3145/3145 [==============================] - 0s 22us/step - loss: nan - acc: 0.7345
Epoch 11/20
3145/3145 [=========================

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?

In [174]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [181]:
df = df.drop(columns="customerID")

X = df.drop(columns="Churn")
y = df["Churn"]

In [182]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit_transform(X)

ValueError: bad input shape (7043, 19)